# BLUE (Best Linear Unbiased Estimator) for Disk Formation
> Created March. 2025 <br>
> Nikhil Bisht<br>

In [1]:
#import modules
import yt
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings
import random
import sys
import time
import imageio
from IPython.display import Image
warnings.filterwarnings("ignore")
import json

from mpl_toolkits.axes_grid1 import AxesGrid


#define paths
SCRATCH = "~/scratch/"
NATURE_RUN = SCRATCH + "/data_assimilation/DiskFormation_9Level_fhundredth_sinkoff/"
LOWRES_RUN = SCRATCH + "/data_assimilation/DiskFormation_3Level_fhundredth_sinkoff/"
ANALYSIS_RUN = SCRATCH + "/data_assimilation/DiskFormation_3Level_analysis/"

NATURE_RUN_INITIAL_CONDITIONS = "./DiskFormation_9Level_fhundredth_sinkoff/DiskFormation_9Level_fhundredth_sinkoff.enzo"
LOWRES_RUN_INITIAL_CONDITIONS = "./DiskFormation_3Level_fhundredth_sinkoff/DiskFormation_3Level_fhundredth_sinkoff.enzo"
ANALYSIS_RUN_INITIAL_CONDITIONS = "./DiskFormation_3Level_analysis/DiskFormation_3Level_analysis.enzo"      #Same as LOWRES RUN

path_to_output_plots = "./plots/"


def make_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def animator(path_to_plots,name_of_gif):
	images = []
	filenames = list(sorted(os.listdir(path_to_plots)))

	for filename in filenames:
	    images.append(imageio.imread(path_to_plots+'/'+filename))
	imageio.mimsave(path_to_plots+'/../'+name_of_gif+'.gif', images)

def run_simulation(path_to_simulation_output, restart_datadump = None, path_to_initial_conditions = None, num_processors = 8):
    if path_to_initial_conditions is not None:
        #copy initial conditions to the simulation output directory and run the simulation there
        curr_dir = os.getcwd()
        os.chdir(path_to_simulation_output)
        os.system("cp "+path_to_initial_conditions+" ./")
        start_time = time.time()
        os.system("mpirun -np "+str(num_processors)+" ./enzo.exe "+path_to_initial_conditions+" |& tee output.txt")
        end_time = time.time()
        os.system("echo 'Simulation took "+str(end_time-start_time)+" seconds' |& tee -a output.txt")
        os.chdir(curr_dir)
        print("Simulation completed. Time taken: "+str(end_time-start_time)+" seconds")
    else:
        if restart_datadump is None:
            print("provide either path to initial conditions or restart datadump")
            return 1    
        #go to the simulation output directory and run the simulation there
        curr_dir = os.getcwd()
        os.chdir(path_to_simulation_output)
        start_time = time.time()
        os.system("mpirun -np "+str(num_processors)+" ./enzo.exe -r "+restart_datadump+" |& tee output.txt")
        end_time = time.time()
        os.system("echo 'Simulation took "+str(end_time-start_time)+" seconds' |& tee -a output.txt")
        os.chdir(curr_dir)
        print("Simulation completed. Time taken: "+str(end_time-start_time)+" seconds")

def simulation_visualize(plot_dir, PlotType, path_to_data, coordinate, dataset_to_plot, number_of_frames,cbar_range = None, zoom_level = 1):
    #import data
    if not os.path.exists(path_to_output_plots+plot_dir+PlotType):
        os.makedirs(path_to_output_plots+plot_dir+PlotType)

    for frame_number in range(0,number_of_frames):
        frame_str = str(frame_number).zfill(4)
        data = yt.load(path_to_data+"DD"+frame_str+"/data"+frame_str)
        all_data = data.all_data()
        plot = yt.ProjectionPlot(data, coordinate, dataset_to_plot, fontsize = 10)
        plot.set_cmap(dataset_to_plot, "dusk")
        plot.set_background_color(dataset_to_plot, color="black")
        plot.zoom(zoom_level)
        plot.annotate_timestamp(corner="upper_left", redshift=True, draw_inset_box=True)
        plot.annotate_scale(corner="upper_right")
        if cbar_range != None:
            plot.set_zlim(dataset_to_plot[1],cbar_range[0],cbar_range[1])
        plot.save(path_to_output_plots+plot_dir+PlotType+"/DD"+frame_str+".png")
        plot.close()
    
    animator(path_to_output_plots+plot_dir+PlotType,PlotType)
    Image(filename=path_to_output_plots+PlotType+'.gif')


